In [ ]:
import json
import pandas as pd


with open('/content/extended_fresh_modilipi_translation.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


df = pd.DataFrame(data)


print(df.head())


  source marathi english
0      𑘀       अ       A
1      𑘁       आ      Aa
2      𑘂       इ       I
3      𑘃       ई      Ee
4      𑘄       उ       U


In [ ]:

df.dropna(inplace=True)


df.rename(columns={'source': 'modilipi', 'marathi': 'marathi', 'english': 'english'}, inplace=True)


print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   modilipi  252 non-null    object
 1   marathi   252 non-null    object
 2   english   252 non-null    object
dtypes: object(3)
memory usage: 6.0+ KB
None


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)


train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)
test.to_csv("test.csv", index=False)


In [ ]:
from transformers import MarianMTModel, MarianTokenizer


model_name = "Helsinki-NLP/opus-mt-mr-en"  # Marathi to English model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/305M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import Dataset

# Convert data to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

def tokenize_function(examples):
    return tokenizer(examples["marathi"], text_target=examples["english"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/201 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import TrainingArguments, Trainer

# Training parameters
training_args = TrainingArguments(
    output_dir="./modilipi_translation",
    # Use 'eval_steps' to control evaluation frequency if 'evaluation_strategy' is not available.
    # For example, evaluate every 500 steps:
    # eval_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maliksourabh2208 (maliksourabh2208-vasantdada-patil-pratishthan-s-college-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61673]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=78, training_loss=0.06431827178368202, metrics={'train_runtime': 3759.6126, 'train_samples_per_second': 0.16, 'train_steps_per_second': 0.021, 'total_flos': 81762856206336.0, 'train_loss': 0.06431827178368202, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.04995594173669815,
 'eval_runtime': 67.1949,
 'eval_samples_per_second': 0.372,
 'eval_steps_per_second': 0.06,
 'epoch': 3.0}

In [ ]:
def translate_modilipi(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Example translation
modilipi_text = "हात स्वच्छ धुवा"
print(translate_modilipi(modilipi_text))


▪ Wash clean water


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#saving the model
model_save_path = '/content/drive/MyDrive/modilipi_translation_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

NameError: name 'trainer' is not defined

In [ ]:
#load the model whenever needed
model_save_path = '/content/drive/MyDrive/modilipi_translation_model'
tokenizer = MarianTokenizer.from_pretrained(model_save_path)
model = MarianMTModel.from_pretrained(model_save_path)

NameError: name 'MarianTokenizer' is not defined

In [ ]:
#reusing the model
from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/modilipi_translation_model'
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

Mounted at /content/drive


NameError: name 'MarianTokenizer' is not defined

In [ ]:
def translate_text(text):
       inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
       outputs = model.generate(**inputs)
       return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [ ]:
text_to_translate = "𑘟𑘝𑘿𑘚𑘰 𑘓𑘿𑘝𑘰 𑘝𑘿𑘙𑘲𑘛𑘂"
translate_text = translate_text(text_to_translate)
print(translate_text)

that was the last time I went to the doctor's office
